In [1]:
import sqlite3
con = sqlite3.connect("phones.db")
cur = con.cursor()

In [2]:
erDiagram


    movie {
        TEXT name PK
        TEXT director
        TEXT country FK
        INTEGER year
        REAL rating

    }

    l_country {
        TEXT country PK
    }

    director {
        TEXT name PK
        INTEGER year
        TEXT country
    }

    l_genres {
        TEXT genre PK
    }

    movie_genre {
        TEXT movie FK
        TEXT genre FK
    }


    movie }|--|| director : "made by"
    movie }|--|| l_country : runs
    movie ||--|{ movie_genre : "sold in"
    l_genres ||--|{ movie_genre : refers

IndentationError: unexpected indent (560920834.py, line 4)

In [3]:
from enum import Enum
class Country(Enum):
    UA = 'Ukraine'
    US = 'United States'
    DE = 'Germany'
    BR = 'Brazil'
    IN = 'India'

    @classmethod
    def valueTuples(self):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        """This function does two things. First, creates a l_platforms table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_country (platform TEXT PRIMARY KEY)')
        # Invoke executemany to insert multiple values at once
        cur.executemany('INSERT INTO l_country VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_country')
        con.commit()
        
    @classmethod
    def getAll(self):
        """Fetch all records from l_platforms table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_country')
        sqlResult = cur.fetchall()
        # Convert tuples in sqlResult to strings and then to enums
        return [Country(p[0]) for p in sqlResult]

In [4]:
Country.removeTable()
Country.createTable()
Country.getAll()

[<Country.UA: 'Ukraine'>,
 <Country.US: 'United States'>,
 <Country.DE: 'Germany'>,
 <Country.BR: 'Brazil'>,
 <Country.IN: 'India'>]

In [5]:
class Genre(Enum):
    comedy = 'Comedy'
    horror = 'Horror'
    thriller = 'Thriller'
    fantasy = 'Fantasy'
    science_fiction = 'Science fiction'

    @classmethod
    def valueTuples(self):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in Genre]

    @classmethod
    def createTable(self):
        """This function does two things. First, creates a l_markets table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre text)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', Genre.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(self):
        """Fetch all records from l_markets table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        # Convert tuples in sqlResult to strings and then to enums
        return [Genre(p[0]) for p in sqlResult]

In [6]:
Genre.removeTable()
Genre.createTable()
Genre.getAll()

[<Genre.comedy: 'Comedy'>,
 <Genre.horror: 'Horror'>,
 <Genre.thriller: 'Thriller'>,
 <Genre.fantasy: 'Fantasy'>,
 <Genre.science_fiction: 'Science fiction'>]

In [7]:
def createGenreMapping():
    cur.execute('''CREATE TABLE MOVIE_GENRE (
                    movie text,
                    genre text,
                    PRIMARY KEY(movie, genre),
                    FOREIGN KEY(movie) REFERENCES movies(name),
                    FOREIGN KEY(genre) REFERENCES l_genres(genre))''')
    con.commit()

In [8]:
createGenreMapping()

In [9]:
class Movie:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        """There is 1-to-1 mapping between phone and its platform,
        therefore we add a FOREIGN KEY to phones table
        """
        cur.execute('''CREATE TABLE movies (
                        name text PRIMARY KEY,
                        director text,
                        country text,
                        year integer,
                        rating REAL,
                        FOREIGN KEY(country) REFERENCES l_country(country))''')
        con.commit()

    def insert(self):
        """There is 1-to-many mapping between phones and markets,
        therefore we have to insert relevant into the above-mentioned phone_market link table
        """
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?, ?)''', (self.name, self.director, self.country, self.year, self.rating))
        # In the list comprehension below, we duplicate phone name, only markets are different
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, m.value) for m in self.genres])
        con.commit()

    def update(self):
        cur.execute('UPDATE movies SET rating=? WHERE name=?', self.rating, self.name)

    @classmethod
    def get(self, name):
        """This query uses a LEFT JOIN and an aggregate function string_agg 
        to fetch markets for a phone.
        """
        cur.execute('''SELECT p.director, p.country, p.year, p.rating, group_concat(pm.genre, ',') FROM movies p
                    LEFT JOIN movie_genre pm ON p.name = pm.movie
                    WHERE p.name=?''', (name,))
        res = cur.fetchone()
        return Movie(name, res[0], res[1], res[2], res[3], res[4])

    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)

In [10]:
Movie.createTable()

In [11]:
movie1 = Movie('Alien: Covenant', 'Ridley Scott', 'USA', 2017, 6.4, {Genre.horror})
movie2 = Movie('Prometheus', 'Ridley Scott', 'USA', 2012, 7.1, {Genre.science_fiction})
movie3 = Movie('Joker', 'Todd Phillips', 'USA', 2019, 8.4, {Genre.thriller})

movie1.insert()
movie2.insert()
movie3.insert()

In [12]:
print(Movie.get('Prometheus'))

(Prometheus, Ridley Scott, USA, 2012, 7.1, Science fiction)


In [13]:
class Director:
    def __init__(self, name, year, country):
        self.name = name
        self.year = year
        self.country = country

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        year INTEGER,
                        country TEXT)''')
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.year, self.country))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET country=? WHERE name=?''', (self.year, self.name))
        con.commit() 

    @classmethod
    def get(self, name):
        cur.execute('SELECT year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        return Director(name, res[0], res[1])

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.year, self.country)

In [14]:
Director.removeTable()
Director.createTable()
Director.getAll()

[]

In [15]:
director1 = Director('Ridley Scott', 1937, 'USA')
director2 = Director('Todd Phillips', 1970, 'USA')
director3 = Director('Stanley Kubrick', 1928, 'USA')

director1.insert()
director2.insert()
director3.insert()

In [16]:
directorFromDb = Director.get('Ridley Scott')
print(directorFromDb)

(Ridley Scott, 1937, USA)


In [17]:
[print(e) for e in Director.getAll()]

(Ridley Scott, 1937, USA)
(Todd Phillips, 1970, USA)
(Stanley Kubrick, 1928, USA)


[None, None, None]